In [ ]:
import torch
torch.cuda.is_available()
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import keras
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader, ConcatDataset
import random
import numpy as np
batch_size=8
from sklearn.metrics import precision_score, recall_score, f1_score
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
# Define the model architecture
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Define the iCaRL class
class iCaRL:
    def __init__(self, device, num_classes, batch_size, memory_size):
        self.device = device
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.memory_size = memory_size
        self.model = CNN(num_classes).to(device)
        self.exemplar_sets = []

    def train(self, dataset, lr, num_epochs):
        train_loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-5)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[49, 63], gamma=0.2)

        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            scheduler.step()
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

    def test(self, dataset, lr, num_epochs):
        dataset = [(transform(inputs), labels) for inputs, labels in tqdm(dataset)]
        test_loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
        criterion = nn.CrossEntropyLoss()
        for epoch in range(num_epochs):
            self.model.eval()
            running_loss = 0.0
            test_loss = 0
            correct = 0
            all_preds = []
            all_labels = []
            batch_size = 4
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                pred = outputs.max(1)[1]
                correct += pred.eq(labels).sum().item()
                preds = pred.cpu().numpy()
                preds = torch.from_numpy(preds).float().to(device)
                all_preds.extend(preds)
                all_labels.extend(labels)
                running_loss += loss.item()
                test_loss = running_loss / len(dataset)
            # Calculate metrics
            precision = precision_score(all_labels, all_preds, average='weighted')
            recall = recall_score(all_labels, all_preds, average='weighted')
            f1 = f1_score(all_labels, all_preds, average='weighted')

            print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(test_loss, correct, len(dataset), 100. * correct / len(dataset)))
            print('Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}\n'.format(precision, recall, f1))

            return 100. * correct / len(labels)


    def construct_exemplar_set(self, dataset, m):
        exemplar_set = []
        class_means = []
        for class_idx in range(self.num_classes):
            class_indices = [idx for idx, (data, label) in enumerate(dataset) if np.array(label) == np.array(class_idx)]
            random.shuffle(class_indices)
            class_indices = class_indices[:m]

            features = []
            for idx in class_indices:
                img, _ = dataset[idx]  # Access data and label directly from the dataset
                img = img.numpy().astype(np.float32) / 255.0
                img = torch.FloatTensor(img).unsqueeze(0).to(self.device)
                feature = self.model.features(img).cpu().squeeze().detach().numpy()
                features.append(feature)
            features = np.array(features)
            class_mean = np.mean(features, axis=0)

            exemplar_set.append(class_mean / np.linalg.norm(class_mean))
            class_means.append(class_mean / np.linalg.norm(class_mean))

        self.exemplar_sets.append(exemplar_set)
        self.class_means = class_means

    def reduce_exemplar_set(self, m):
        for exemplar_set in self.exemplar_sets:
            exemplar_set = exemplar_set[:m]

In [ ]:
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split

data_dir = '/content/drive/MyDrive/IIIT/Various_datasets/Satelite Image Classification/OG data/Original Data'

def create_train_test_datasets(dataset_dir, split_size):
    # Create ImageFolder dataset
    dataset = datasets.ImageFolder(root=dataset_dir, transform=None)

    # Calculate sizes for train and test datasets
    train_size = int(len(dataset) * split_size)
    test_size = len(dataset) - train_size

    # Split dataset into train and test datasets
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    return train_dataset, test_dataset

# Define transformations to convert PIL images to tensors
transform = transforms.Compose([transforms.Resize(32),
    transforms.ToTensor()
    # Add more transformations as needed
])

# Example usage:
split_size = 0.8  # 80% train, 20% test
dataset = datasets.ImageFolder(root=data_dir, transform=None)
train_size = int(len(dataset) * split_size)
test_size = len(dataset) - train_size

# train_dataset, test_dataset = create_train_test_datasets(data_dir, split_size)
train_data, test_data = random_split(dataset, [train_size, test_size])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
num_classes = 4
batch_size = 8
memory_size = 2000
lr = 0.0005
num_epochs = 50
m = memory_size // num_classes

# Initialize iCaRL
icarl = iCaRL(device, num_classes, batch_size, memory_size)

# Define the number of classes to train on for each task
classes_per_task = 2  # You can set this to any number you prefer

from torchvision import transforms

num_tasks = (num_classes + classes_per_task - 1) // classes_per_task
train_datasets = []

# Iterate through each task
for task_idx in range(num_tasks):
    # Calculate the range of classes for the current task
    start_class_idx = task_idx * classes_per_task
    end_class_idx = min(start_class_idx + classes_per_task, num_classes)

    # Filter indices based on the desired class range
    indices = [idx for idx in range(len(train_data)) if start_class_idx <= train_data.dataset.targets[train_data.indices[idx]] < end_class_idx]

    # Create a subset of the original dataset for the current task
    subset = Subset(train_data.dataset, indices)

    # Apply transformations to the subset
    subset = [(transform(sample), target) for sample, target in tqdm(subset)]

    # Add the subset to the list of train datasets
    train_datasets.append(subset)


# Incremental training
for i, train_dataset in enumerate(train_datasets):
    print(f'Training on task {i + 1}')
    icarl.train(train_dataset, lr, num_epochs)
    print(f'Constructing exemplar set for task {i + 1}')
    icarl.construct_exemplar_set(train_dataset, m)
    if i > 0:
        print(f'Reducing exemplar set for task {i}')
        icarl.reduce_exemplar_set(m)

# Evaluation
with torch.no_grad():
  icarl.test(test_data, lr, num_epochs)

  0%|          | 0/955 [00:00<?, ?it/s]

  0%|          | 0/897 [00:00<?, ?it/s]

Training on task 1
Epoch [1/50], Loss: 1.3762
Epoch [2/50], Loss: 1.3302
Epoch [3/50], Loss: 1.2718
Epoch [4/50], Loss: 1.2216
Epoch [5/50], Loss: 1.1862
Epoch [6/50], Loss: 1.1562
Epoch [7/50], Loss: 1.1384
Epoch [8/50], Loss: 1.1181
Epoch [9/50], Loss: 1.0960
Epoch [10/50], Loss: 1.0839
Epoch [11/50], Loss: 1.0594
Epoch [12/50], Loss: 1.0392
Epoch [13/50], Loss: 1.0214
Epoch [14/50], Loss: 0.9988
Epoch [15/50], Loss: 0.9825
Epoch [16/50], Loss: 0.9562
Epoch [17/50], Loss: 0.9329
Epoch [18/50], Loss: 0.8977
Epoch [19/50], Loss: 0.8726
Epoch [20/50], Loss: 0.8487
Epoch [21/50], Loss: 0.8106
Epoch [22/50], Loss: 0.7888
Epoch [23/50], Loss: 0.7560
Epoch [24/50], Loss: 0.7275
Epoch [25/50], Loss: 0.7040
Epoch [26/50], Loss: 0.6834
Epoch [27/50], Loss: 0.6603
Epoch [28/50], Loss: 0.6390
Epoch [29/50], Loss: 0.6197
Epoch [30/50], Loss: 0.5968
Epoch [31/50], Loss: 0.5828
Epoch [32/50], Loss: 0.5755
Epoch [33/50], Loss: 0.5485
Epoch [34/50], Loss: 0.5441
Epoch [35/50], Loss: 0.5234
Epoch [36/

  0%|          | 0/463 [00:00<?, ?it/s]

Test set: Average loss: 0.0055, Accuracy: 318/463 (69%)
Precision: 0.5327, Recall: 0.6868, F1 Score: 0.5902



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
